In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import requests
from bs4 import BeautifulSoup
from glob import glob
from time import sleep
from os.path import isfile
from weasyprint import HTML
from urllib.parse import urljoin
from tqdm.autonotebook import tqdm

/home/.local/lib/python3.6/site-packages/ipykernel_launcher.py:8: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  


In [3]:
url = 'https://brunch.co.kr/@begini'

path = 'BAEL LEATHER SCHOOL'   

with open('data/blog/BAEL LEATHER SCHOOL.html', 'r') as fp:
    list_soup = BeautifulSoup(''.join(fp.readlines()))
    
list_soup

<html><body><ul class="list_article list_post1 #post_list">
<li class="animation_up" data-articleuid="23zo_769">
<a class="link_category" href="/magazine/bagmaking">
<em class="tit_category">
                이탈리아 방식의 가죽 가방 만들기
            </em>
</a>
<a class="link_post has_image #post_listview" href="/@begini/769">
<strong class="tit_subject">작업실 임대 놓습니다</strong>
<div class="post_thumb">
<img alt="이미지정보" class="img_thumb" src="//i1.daumcdn.net/thumb/C120x120/?fname=http://t1.daumcdn.net/brunch/service/user/23zo/image/TVYJQoUuxgUnkiMfKA13uwwDAkk.jpg"/>
</div>
<div class="post_desc">
<div class="wrap_sub_content">
<em class="tit_sub"></em>
<span class="article_content">제가 작업실을 후암동으로 이전하게 되어서현재 면목동 작업실을 임대 놓고자 합니다.여러가지 집기들은 협의 가능하세요.가죽공예를
                        하려면 어느정도 공간도 필요하고소음도 신경쓰이는데요.지금 공간이면 문제 없을듯 하고요.개인 또는 두 세분이 쉐어도 좋을듯요.좋은 분이 인수하셨으면 합니다.감사합니다.위치:
                        면목시장 주차장 앞(7호선 용마산역 도보 </span>
</div>
</div>
<span class="post_append">
<span>공유</span>
<span class="num_txt">

In [4]:
article_list = [urljoin(url, x['href']) for x in list_soup.select('li a') if x.has_attr('href') if x['href'].find('magazine') < 0]

article_list = list(set(article_list))
article_list

['https://brunch.co.kr/@begini/179',
 'https://brunch.co.kr/@begini/675',
 'https://brunch.co.kr/@begini/142',
 'https://brunch.co.kr/@begini/618',
 'https://brunch.co.kr/@begini/137',
 'https://brunch.co.kr/@begini/255',
 'https://brunch.co.kr/@begini/125',
 'https://brunch.co.kr/@begini/559',
 'https://brunch.co.kr/@begini/418',
 'https://brunch.co.kr/@begini/531',
 'https://brunch.co.kr/@begini/674',
 'https://brunch.co.kr/@begini/89',
 'https://brunch.co.kr/@begini/586',
 'https://brunch.co.kr/@begini/92',
 'https://brunch.co.kr/@begini/34',
 'https://brunch.co.kr/@begini/170',
 'https://brunch.co.kr/@begini/769',
 'https://brunch.co.kr/@begini/128',
 'https://brunch.co.kr/@begini/564',
 'https://brunch.co.kr/@begini/543',
 'https://brunch.co.kr/@begini/694',
 'https://brunch.co.kr/@begini/615',
 'https://brunch.co.kr/@begini/665',
 'https://brunch.co.kr/@begini/679',
 'https://brunch.co.kr/@begini/189',
 'https://brunch.co.kr/@begini/163',
 'https://brunch.co.kr/@begini/592',
 'ht

In [5]:
data_path = 'data/blog/{}'.format(path)

f_index = {}
for f in glob('{}/*.pdf'.format(data_path)):
    doc_id = int(f.rsplit('/')[-1].split('-')[0])
    f_index[doc_id] = True
    
print(f_index)

{527: True, 614: True, 623: True, 353: True, 549: True, 610: True, 474: True, 538: True, 508: True, 558: True, 194: True, 414: True, 26: True, 472: True, 260: True, 396: True, 571: True, 497: True, 368: True, 693: True, 621: True, 71: True, 611: True, 446: True, 225: True, 493: True, 665: True, 514: True, 179: True, 232: True, 27: True, 201: True, 683: True, 439: True, 676: True, 552: True, 518: True, 369: True, 662: True, 584: True, 107: True, 426: True, 158: True, 18: True, 133: True, 113: True, 677: True, 450: True, 157: True, 535: True, 551: True, 672: True, 380: True, 397: True, 475: True, 459: True, 510: True, 617: True, 186: True, 35: True, 48: True, 448: True, 363: True, 433: True, 618: True, 175: True, 682: True, 661: True, 416: True, 452: True, 645: True, 516: True, 465: True, 142: True, 559: True, 378: True, 575: True, 398: True, 542: True, 252: True, 541: True, 536: True, 447: True, 696: True, 434: True, 132: True, 622: True, 202: True, 83: True, 531: True, 550: True, 189: 

In [6]:
def restore_url(soup):
    for tag in soup.find_all():
        if tag.has_attr('src') is False:
            continue

        tag['src'] = urljoin(url, tag['src'])  
        
    return soup

In [7]:
def save_article(soup):
    title = ''.join([x.get_text().strip() for x in soup.select('div.cover_cell')])
    filename = '{}/{:04d}-{}.pdf'.format(data_path, doc_id, title.replace('/', '-'))
    
    body = '\n'.join([x.prettify() for x in soup.select('div.wrap_body')])

    HTML(string=body).write_pdf(filename)   
    
    with open(filename.replace('.pdf', '.html'), 'w') as fp:
        fp.write(body)
        
    return

In [8]:
pbar = tqdm(article_list)
for c_url in pbar:
    doc_id = int(c_url.split('/')[-1])
    pbar.set_description(desc=str(doc_id))
    
    if doc_id in f_index:
        continue
    
    resp = requests.get(c_url)

    soup = BeautifulSoup(resp.text)
    soup = restore_url(soup)
    
    save_article(soup=soup)    
    sleep(5)